# Instalación de librerías

Lo primero es instalar las librerías necesarias para realizar la tarea.

In [16]:
# Instalamos las librerías necesarias
%pip install scrapy bs4 elasticsearch==8.12.1 sklearn nltk

Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'd:\JuanluHijo\Programs\python3102\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Juanlu hijo\\AppData\\Local\\Temp\\pip-install-6jj8qsdi\\sklearn_bcc23bc52a934c5cb046915a27624a01\\setup.py'"'"'; __file__='"'"'C:\\Users\\Juanlu hijo\\AppData\\Local\\Temp\\pip-install-6jj8qsdi\\sklearn_bcc23bc52a934c5cb046915a27624a01\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Juanlu hijo\AppData\Local\Temp\pip-pip-egg-info-npqo7ea9'
         cwd: C:\Users\Juanlu hijo\AppData\Local\Temp\pip-install-6jj8qsdi\sklearn_bcc23bc52a934c5cb046915a27624a01\
    Complete output (15 lines):
    The 'sklearn' PyPI package is deprecated, use 'scikit-learn'

# Compilar datos del documento web

Se va a crear un Crawler para extraer información de la página de Bulbapedia. Conretamente, el crawler va a extraer la información de todos lo Pokémon, empezará por un primer enlace (Abomasnow) y seguirá con los enlaces de la página para obtener la información del resto. 

Vamos a crear este crawler en un archivo `.py` a parte ([`crawler.py`](crawler.py#L15)).

Una vez tenemos creado el crawler, vamos a ejecutarlo para obtener los datos. Los resultados de cada pokemon se guardarán en un archivo JSON cuyo nombre será el número de la pokédex del pokemon.

In [2]:
from crawler import PokedexSpyder
import scrapy, os
from scrapy.crawler import CrawlerProcess
import nest_asyncio
nest_asyncio.apply()


# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Comprobamos que existe la carpeta y si no existe la creamos
if not os.path.exists('pokedex'):
    os.mkdir('pokedex')

# Creamos el proceso con el RSSSpider
process.crawl(PokedexSpyder)

# Ejecutamos el Crawler
process.start()

Se han encontrado más de 500 pokemons, se para el crawler


# Buscador

Importamos las librerías necesarias.

In [3]:
from elasticsearch import Elasticsearch, helpers
import uuid

Nos conectamos a Elasticsearch.

In [4]:
from elasticsearch import Elasticsearch

client = Elasticsearch(['http://localhost:9200'])

def obtener_informacion_cluster():
    try:
        info_cluster = client.cluster.health()
        print("Información del clúster:")
        print(info_cluster)
    except Exception as e:
        print("Error al obtener información del clúster:", e)

obtener_informacion_cluster()

Información del clúster:
{'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 25, 'active_shards': 25, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


Creamos el índice.

In [5]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    [{'host': 'localhost', 'port': 9200, 'scheme': 'http'}],
    verify_certs=False
)
index_name = "pokemon_index"

# 1) Borra el índice si ya existe
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# 2) Crea el índice con el analyzer completo
es.indices.create(
    index=index_name,
    body={
      "settings": {
        "analysis": {
          "filter": {
            "spanish_stop": {
              "type":       "stop",
              "stopwords":  "_spanish_"
            },
            "spanish_stemmer": {
              "type":     "stemmer",
              "language": "light_spanish"
            }
          },
          "analyzer": {
            "my_spanish": {
              "tokenizer": "standard",
              "filter": [
                "lowercase",
                "spanish_stop",
                "spanish_stemmer"
              ]
            }
          }
        }
      },
      "mappings": {
        "dynamic_templates": [
          {
            "descriptions_as_spanish_text": {
              "path_match": "descriptions.*",
              "mapping": {
                "type":     "text",
                "analyzer": "my_spanish",
                "copy_to":  "all_descriptions"
              }
            }
          }
        ],
        "properties": {
          "name":             {"type":"keyword"},
          "number":           {"type":"integer"},
          "url":              {"type":"text"},
          "types":            {"type":"keyword"},
          "class":            {"type":"text"},
          "weight":           {"type":"float"},
          "height":           {"type":"float"},
          "all_descriptions": {
                               "type":     "text",
                               "analyzer": "my_spanish"
                             },
          "descriptions":     {"type":"object","dynamic":True}
        }
      }
    }
)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'pokemon_index'})

Cargamos los datos en Elasticsearch.

In [6]:
import json
import os
from pathlib import Path
from pprint import pprint

# Creamos una lista para guardar los documentos
documents: list = [] 

# Directorio donde se encuentran los archivos JSON
directory: Path = Path('pokedex')

# Eliminamos todos los documentos ya subidos a elasticsearch con el índice "index_name"
es.delete_by_query(
    index=index_name,
    body={
        "query": {
            "match_all": {}
        }
    }
)

# Creamos un bucle que recorra los archivos JSON de la carpeta pokedex
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        file_path = directory / filename
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            document = {
                "_index": index_name,
                "_id": uuid.uuid4(),
                "_source": {
                    "url": data["url"],
                    "number": data["number"],
                    "name": data["name"],
                    "types": data["types"],
                    "class": data["class"],
                    "height": data["height"],
                    "weight": data["weight"],
                    "descriptions": data["descriptions"],
                },
            }
            documents.append(document)

# Vemos un documento de ejemplo
# pprint(documents[0])

# Insertamos los documentos en Elasticsearch
helpers.bulk(es, documents)

(501, [])

In [7]:
import time

# Comprobamos que se han insertado los documentos
time.sleep(1)
count = es.count(index=index_name)['count']
print(f"Number of documents in index '{index_name}': {count}")

# Visualizamos un documento
response = es.search(index=index_name, body={"query": {"match_all": {}}})
print("Primer documento:")
pprint(response['hits']['hits'][0]['_source'])

Number of documents in index 'pokemon_index': 501
Primer documento:
{'class': 'Pokémon Semilla',
 'descriptions': {' Pokémon Amarillo': 'Puede sobrevivir largo tiempo sin '
                                       'probar bocado. Guarda energía en el '
                                       'bulbo de su espalda',
                  ' Pokémon Blanco': 'Después de nacer, crece alimentándose de '
                                     'la semilla de su lomo.',
                  ' Pokémon Cristal': 'Cuando es joven, crece con los '
                                      'nutrientes que almacena en las semillas '
                                      'de su espalda.',
                  ' Pokémon Diamante': 'Después de nacer, crece alimentándose '
                                       'de las semillas de su lomo.',
                  ' Pokémon Esmeralda': 'A Bulbasaur es fácil verle echándose '
                                        'una siesta al sol. La semilla que '
                           

## AND, OR y ordenar resultados por número

In [8]:
types = ["bicho", "planta"] # lista de 1 o 2 elementos
match_type = "and" # Elegitr entre "and" o "or"

query = {
    "size": 500,
    "query": {
        "bool": {
            "should": [
                {"term": {"types": t}} for t in types
            ],
            "minimum_should_match": 1 if match_type == "or" else len(types)
        }
    },
    "sort": [
        {"number": {"order": "asc"}}
    ],
}

resp = es.search(index=index_name, body=query)
print(f"Encontrados {resp['hits']['total']['value']} pokémon ({match_type.upper()}) de tipo {types}:")
print()
for hit in resp["hits"]["hits"]:
    print(f"- {hit['_source']['name']}: {hit['_source']['types']}")


Encontrados 6 pokémon (AND) de tipo ['bicho', 'planta']:

- Paras: ['bicho', 'planta']
- Parasect: ['bicho', 'planta']
- Wormadam: ['bicho', 'planta']
- Sewaddle: ['bicho', 'planta']
- Swadloon: ['bicho', 'planta']
- Leavanny: ['bicho', 'planta']


## NOT

In [9]:
# Lista de tipos que queremos excluir
exlude_types = ["fuego", "agua", "hielo"]

query = {
    "size": 501,
    "query": {
        "bool": {
            "must_not": [
                {
                    "terms": {
                        "types": exlude_types
                    }
                }
            ]
        }
    },
    # Opcional: ordenar por número ascendente
    "sort": [
        {"number": {"order": "asc"}}
    ]
}

resp = es.search(index=index_name, body=query)
print(f"Encontrados {resp['hits']['total']['value']} pokémon que no contienen los tipos {exlude_types}:")
print()
for hit in resp["hits"]["hits"]:
    print(f"- {hit['_source']['name']}: {hit['_source']['types']}")

Encontrados 402 pokémon que no contienen los tipos ['fuego', 'agua', 'hielo']:

- Bulbasaur: ['planta', 'veneno']
- Ivysaur: ['planta', 'veneno']
- Venusaur: ['planta', 'veneno']
- Caterpie: ['bicho']
- Metapod: ['bicho']
- Butterfree: ['bicho', 'volador']
- Weedle: ['bicho', 'veneno']
- Kakuna: ['bicho', 'veneno']
- Beedrill: ['bicho', 'veneno']
- Pidgey: ['normal', 'volador']
- Pidgeotto: ['normal', 'volador']
- Pidgeot: ['normal', 'volador']
- Spearow: ['normal', 'volador']
- Fearow: ['normal', 'volador']
- Nidorina: ['veneno']
- Nidoqueen: ['veneno', 'tierra']
- Nidorino: ['veneno']
- Nidoking: ['veneno', 'tierra']
- Clefairy: ['hada']
- Clefable: ['hada']
- Jigglypuff: ['normal', 'hada']
- Wigglytuff: ['normal', 'hada']
- Zubat: ['veneno', 'volador']
- Golbat: ['veneno', 'volador']
- Oddish: ['planta', 'veneno']
- Gloom: ['planta', 'veneno']
- Vileplume: ['planta', 'veneno']
- Paras: ['bicho', 'planta']
- Parasect: ['bicho', 'planta']
- Venonat: ['bicho', 'veneno']
- Venomoth: ['b

## Buscar en un rango

In [10]:
# Ejemplos de entrada:  
# [], no filtro de peso  
# [5.0], sólo filtro ≥ 5.0  
# [None, 10.0], sólo filtro ≤ 10.0  
# [1.0, 20.0], filtro entre 1.0 y 20.0  
weight = [1.0, 5.0]
height = [None, 1.0]  # sólo filtro altura ≤ 1.0

# Construimos lista de filtros
filters = []

# Rango para weight
if weight:
    wmin, wmax = weight if len(weight) == 2 else (weight[0], None)
    rango_w = {}
    if wmin is not None:
        rango_w["gte"] = wmin
    if wmax is not None:
        rango_w["lte"] = wmax
    if rango_w:
        filters.append({"range": {"weight": rango_w}})

# Rango para height
if height:
    hmin, hmax = height if len(height) == 2 else (height[0], None)
    rango_h = {}
    if hmin is not None:
        rango_h["gte"] = hmin
    if hmax is not None:
        rango_h["lte"] = hmax
    if rango_h:
        filters.append({"range": {"height": rango_h}})

# Montamos la query con bool.filter (=> AND de todos los filtros)
query = {
    "query": {
        "bool": {
            "filter": filters
        }
    },
    "sort": [
        {"number": {"order": "asc"}}
    ]
}

resp = es.search(index=index_name, body=query, size=501)
print(f"Encontrados {resp['hits']['total']['value']} pokémon en el rango solicitado:")
for hit in resp["hits"]["hits"]:
    src = hit["_source"]
    print(f"- #{src['number']:>3} {src['name']}: peso={src['weight']} altura={src['height']}")

Encontrados 75 pokémon en el rango solicitado:
- #010 Caterpie: peso=2.9 altura=0.3
- #013 Weedle: peso=3.2 altura=0.3
- #016 Pidgey: peso=1.8 altura=0.3
- #021 Spearow: peso=2.0 altura=0.3
- #069 Bellsprout: peso=4.0 altura=0.7
- #090 Shellder: peso=4.0 altura=0.3
- #102 Exeggcute: peso=2.5 altura=0.4
- #109 Koffing: peso=1.0 altura=0.6
- #151 Mew: peso=4.0 altura=0.4
- #172 Pichu: peso=2.0 altura=0.3
- #173 Cleffa: peso=3.0 altura=0.3
- #174 Igglybuff: peso=1.0 altura=0.3
- #175 Togepi: peso=1.5 altura=0.3
- #176 Togetic: peso=3.2 altura=0.6
- #177 Natu: peso=2.0 altura=0.2
- #188 Skiploom: peso=1.0 altura=0.6
- #189 Jumpluff: peso=3.0 altura=0.8
- #191 Sunkern: peso=1.8 altura=0.3
- #198 Murkrow: peso=2.1 altura=0.5
- #200 Misdreavus: peso=1.0 altura=0.7
- #201 Unown: peso=5.0 altura=0.5
- #211 Qwilfish: peso=3.9 altura=0.5
- #251 Celebi: peso=5.0 altura=0.6
- #265 Wurmple: peso=3.6 altura=0.3
- #276 Taillow: peso=2.3 altura=0.3
- #285 Shroomish: peso=4.5 altura=0.4
- #292 Shedinja:

## Buscar por frase

In [11]:
import re

# La frase que quieres buscar
phrase = "vive en"
pattern = re.compile(re.escape(phrase), re.IGNORECASE)

query = {
    "query": {
        "multi_match": {
            "query":  phrase,
            "type":   "phrase",
            "fields": ["descriptions.*"]
        }
    }
}

resp = es.search(index=index_name, body=query, size=100)
print(f"Encontrados {resp['hits']['total']['value']} pokémon con «{phrase}» en alguna descripción:\n")

for hit in resp["hits"]["hits"]:
    src = hit["_source"]
    # Filtramos solo los juegos cuyas descripciones contengan la frase
    matches = [
        (juego.strip(), texto)
        for juego, texto in src["descriptions"].items()
        if pattern.search(texto)
    ]
    if not matches:
        continue
    print(f"- #{src['number']:>3} {src['name']}:")
    for juego, texto in matches:
        m = pattern.search(texto)
        before = texto[:m.start()]
        match = texto[m.start():m.end()]
        after  = texto[m.end():]
        highlighted = f"{before}<<{match}>>{after}"
        print(f"    {juego}: {highlighted}")
    print()

Encontrados 59 pokémon con «vive en» en alguna descripción:

- #771 Pyukumuku:
    Pokémon Sol: <<Vive en>> las playas y en aguas poco profundas. Expulsa sus entrañas para engañar a sus depredadores y librarse de ellos.

- #467 Magmortar:
    Pokémon Diamante: Sus brazos disparan bolas de fuego de más de 2000 grados. <<Vive en>> cráteres volcánicos.
    Pokémon Perla: Sus brazos disparan bolas de fuego de más de 2000 grados. <<Vive en>> cráteres volcánicos.
    Pokémon X: Sus brazos disparan bolas de fuego de más de 2000 °C. <<Vive en>> cráteres volcánicos.
    Pokémon Rubí Omega: Sus brazos disparan bolas de fuego de más de 2000 °C. <<Vive en>> cráteres volcánicos.
    Pokémon Sol: <<Vive en>> cráteres volcánicos. Al parecer, en cada volcán solo habita una pareja de Magmortar.

- #485 Heatran:
    Pokémon HeartGold: Su sangre fluye ardiendo como si fuera magma. <<Vive en>> cráteres de volcanes.
    Pokémon SoulSilver: Su sangre fluye ardiendo como si fuera magma. <<Vive en>> cráteres 

## Paginación de resultados

In [12]:
phrase = "vive en"
pattern = re.compile(re.escape(phrase), re.IGNORECASE)

# Parámetros de paginación
page_size   = 5    # cuántos resultados por página
page_number = 2    # página a recuperar (1-based)

# Calculamos offset
from_ = (page_number - 1) * page_size

query = {
    "from": from_,
    "size": page_size,
    "query": {
        "multi_match": {
            "query":  phrase,
            "type":   "phrase",
            "fields": ["descriptions.*"]
        }
    },
    "sort": [
        {"number": {"order": "asc"}}
    ]
}

resp = es.search(index=index_name, body=query)
total = resp["hits"]["total"]["value"]
hits  = resp["hits"]["hits"]

print(f"Página {page_number} ({len(hits)} resultados) de { (total + page_size - 1) // page_size } — Total: {total}\n")

for hit in hits:
    src = hit["_source"]
    matches = [
        (juego.strip(), texto)
        for juego, texto in src["descriptions"].items()
        if pattern.search(texto)
    ]
    if not matches:
        continue
    print(f"- #{src['number']:>3} {src['name']}:")
    for juego, texto in matches:
        m = pattern.search(texto)
        before = texto[:m.start()]
        match  = texto[m.start():m.end()]
        after  = texto[m.end():]
        highlighted = f"{before}<<{match}>>{after}"
        print(f"    {juego}: {highlighted}")
    print()


Página 2 (5 resultados) de 12 — Total: 59

- #071 Victreebel:
    Pokémon Rojo y Azul: <<Vive en>> grandes colonias en el interior de las junglas, aunque nadie ha podido verificarlo
    Pokémon Verde Hoja: Dicen que <<vive en>> grandes colonias en el interior de las junglas, aunque nadie ha podido verificarlo.
    Pokémon X: Dicen que <<vive en>> grandes colonias en el interior de las junglas, aunque nadie ha podido verificarlo.

- #073 Tentacruel:
    Pokémon Esmeralda: <<Vive en>> formaciones complejas de roca en el suelo marino y atrapa a su presa usando sus 80 tentáculos. Si se pone nervioso, le brillan las esferas rojas de la cabeza.

- #076 Golem:
    Pokémon Rubí: Golem <<vive en>> las montañas. Si se produce un gran terremoto, estos Pokémon descienden rodando en masa por las laderas.
    Pokémon Zafiro: Golem es conocido por su afición a bajar de las montañas rodando. La gente que <<vive en>> la falda de las mismas ha cavado surcos para conducirlo en su descenso por las laderas

## Uso de stemming

In [13]:
# Frase de búsqueda, puede ser varias palabras
phrase = "alimenta"

# 1) Obtenemos TODOS los stems de la frase usando tu analyzer custom "my_spanish"
an = es.indices.analyze(index=index_name, body={
    "analyzer": "my_spanish",
    "text":     phrase
})

# Extraemos los stems únicos
stems = list({token["token"] for token in an["tokens"]})
print("Stems extraídos:", stems)
# Ejemplo: ['com', 'comiend', 'comerá', 'come', 'comid'] según tu analyzer

# 2) Montamos el regex con todas las raíces: \b(root1\w*|root2\w*|…)\b
pattern = re.compile(
    r"\b(" + "|".join(re.escape(s) + r"\w*" for s in stems) + r")\b",
    re.IGNORECASE
)

# 3) Consulta a Elasticsearch aplicando el mismo analyzer para matching
page_size   = 5
page_number = 1
_from = (page_number - 1) * page_size

resp = es.search(
    index=index_name,
    body={
      "from": _from,
      "size": page_size * 10,   # traemos más docs porque puede haber múltiples matches por doc
      "query": {
        "multi_match": {
           "query":    phrase,
           "fields":   ["descriptions.*"],
           "analyzer": "my_spanish",
           "lenient":  True
        }
      },
      "sort": [{"number": {"order": "asc"}}]
    }
)

# 4) Aplanamos en una lista de coincidencias individuales
matches = []
for hit in resp["hits"]["hits"]:
    src  = hit["_source"]
    num  = src["number"]
    name = src["name"]
    for juego, texto in src["descriptions"].items():
        if pattern.search(texto):
            # resaltamos todas las formas derivadas encontradas
            highlighted = pattern.sub(lambda m: f"<<{m.group(0)}>>", texto)
            matches.append((num, name, juego.strip(), highlighted))

# 5) Paginación sobre esas coincidencias
total_matches = len(matches)
total_pages   = (total_matches + page_size - 1) // page_size
start = (page_number - 1) * page_size
end   = start + page_size

print(f"\nPágina {page_number}/{total_pages} — mostrando coincidencias {start+1}–{min(end, total_matches)} de {total_matches}\n")
for num, name, juego, snippet in matches[start:end]:
    print(f"- #{num:>3} {name} [{juego}]")
    print(f"    {snippet}\n")


Stems extraídos: ['aliment']

Página 1/19 — mostrando coincidencias 1–5 de 92

- #016 Pidgey [Pokémon Rojo Fuego]
    A este Pokémon no le gusta luchar. Suele permanecer escondido en zonas de hierba alta. Se <<alimenta>> de pequeños insectos.

- #030 Nidorina [Pokémon Oro]
    Cuando <<alimenta>> a sus crías, primero mastica y ablanda la comida y luego la escupe para su prole.

- #030 Nidorina [Pokémon HeartGold]
    Cuando <<alimenta>> a sus crías, primero mastica y ablanda la comida y luego la escupe para su prole.

- #030 Nidorina [Pokémon SoulSilver]
    Cuando <<alimenta>> a sus crías, primero mastica y ablanda la comida y luego la escupe para su prole.

- #030 Nidorina [Pokémon Y]
    Cuando <<alimenta>> a sus crías, primero mastica y ablanda la comida y luego la escupe para su prole.



## Busqueda Fuzzy

In [14]:
# Frase a buscar (con tolerancia a errores)
phrase      = "comer"    # por ejemplo, puedes poner "comer", "vivir", etc.
page_size   = 5
page_number = 1
_from       = (page_number - 1) * page_size

# Montamos la query fuzzy + highlight + paginación
query = {
    "from": _from,
    "size": page_size,
    "query": {
        "multi_match": {
            "query":        phrase,
            "fields":       ["descriptions.*"],
            "type":         "best_fields",
            "fuzziness":    "AUTO",    # activa fuzzy
            "prefix_length": 2,        # primeros X caracteres exactos
            "max_expansions": 50,      # variantes a generar
            "operator":     "and"      # todas las palabras deben coincidir (si son varias)
        }
    },
    "highlight": {
        "pre_tags":  ["<<"],
        "post_tags": [">>"],
        "fields": {
            "descriptions.*": {}     # highlight en todos los subcampos
        }
    },
    "sort": [
        {"number": {"order": "asc"}}
    ]
}

# Ejecutamos la búsqueda
resp = es.search(index=index_name, body=query)
total = resp["hits"]["total"]["value"]
hits  = resp["hits"]["hits"]

# Cálculo de páginas
total_pages = (total + page_size - 1) // page_size

print(f"Página {page_number}/{total_pages} — resultados { _from+1 }–{_from+len(hits)} de {total} para «{phrase}» (fuzzy)\n")

# Imprimimos cada hit con su highlight
for hit in hits:
    src        = hit["_source"]
    highlights = hit.get("highlight", {})
    if not highlights:
        continue
    print(f"- #{src['number']:>3} {src['name']}")
    # Cada campo "descriptions.<juego>"
    for field, frags in highlights.items():
        juego = field.split(".",1)[1]
        for frag in frags:
            print(f"    {juego.strip()}: {frag}")
    print()


Página 1/8 — resultados 1–5 de 39 para «comer» (fuzzy)

- #013 Weedle
    Pokémon Negro: <<Come>> el equivalente a su peso en hojas todos los días. Se defiende con el aguijón de su cabeza.
    Pokémon Blanco: <<Come>> el equivalente a su peso en hojas todos los días. Se defiende con el aguijón de su cabeza.
    Pokémon Perla: <<Come>> el equivalente a su peso en hojas todos los días. Se defiende con el aguijón de su cabeza.
    Pokémon Diamante: <<Come>> el equivalente a su peso en hojas todos los días. Se defiende con el aguijón de su cabeza.
    Pokémon Plata: Suele encontrarse debajo de las hojas que <<come>>.
    Pokémon Amarillo: Se esconde en la hierba y arbustos mientras <<come>>
    Pokémon Platino: <<Come>> el equivalente a su peso en hojas todos los días. Se defiende con el aguijón de su cabeza.

- #021 Spearow
    Pokémon Rojo y Azul: <<Come>> bichos en zonas de hierba. Agita sus cortas alas muy rápido para mantenerse en el aire
    Pokémon Luna: Tiene un apetito voraz y <<c

## Uso de term suggester

In [15]:
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host':'localhost','port':9200,'scheme':'http'}], verify_certs=False)
index_name = "pokemon_index"

phrase      = "cojer"   # ejemplo con typo
page_size   = 5
page_number = 1
_from       = (page_number - 1) * page_size

body = {
  # 8) Term suggester sobre el campo all_descriptions
  "suggest": {
    "spell_correction": {
      "text": phrase,
      "term": {
        "field": "all_descriptions",
        "suggest_mode": "always",
        "min_word_length": 3
      }
    }
  },
  "from": _from,
  "size": page_size,
  "query": {
    "multi_match": {
      "query":        phrase,
      "fields":       ["descriptions.*"],
      "type":         "best_fields",
      "fuzziness":    "AUTO",
      "prefix_length":2,
      "max_expansions":50,
      "operator":     "and"
    }
  },
  "highlight": {
    "pre_tags":  ["<<"],
    "post_tags": [">>"],
    "fields": {
      "descriptions.*": {}
    }
  },
  "sort": [
    {"number": {"order":"asc"}}
  ]
}

resp = es.search(index=index_name, body=body)

# 1) Mostrar sugerencias
for sug in resp.get("suggest", {}).get("spell_correction", []):
    for opt in sug.get("options", []):
        print(f"¿Quisiste decir? {opt['text']} (score {opt['score']:.2f})")

print()

# 2) Mostrar resultados fuzzy + highlight
for hit in resp["hits"]["hits"]:
    src        = hit["_source"]
    highlights = hit.get("highlight", {})
    if not highlights:
        continue
    print(f"#{src['number']} {src['name']}")
    for field, frags in highlights.items():
        juego = field.split(".",1)[1]
        for frag in frags:
            print(f"  {juego}: {frag}")
    print()


¿Quisiste decir? comer (score 0.80)
¿Quisiste decir? coger (score 0.80)
¿Quisiste decir? coler (score 0.80)
¿Quisiste decir? cocer (score 0.80)
¿Quisiste decir? color (score 0.60)

#022 Fearow
   Pokémon Plata: Usa sabiamente su fino y largo pico para extraer y <<comer>> insectos que se ocultan bajo tierra.

#065 Alakazam
   Pokémon Cristal: Lo analiza todo al detalle para <<coger>> ventaja en los combates.

#075 Graveler
   Pokémon Sol: Llega a <<comer>> más de una tonelada en un día y causa un gran estruendo al masticar.

#089 Muk
   Pokémon Zafiro Alfa: A este Pokémon le encanta <<comer>> cosas repulsivas.
   Pokémon Zafiro: A este Pokémon le encanta <<comer>> cosas repulsivas.

#107 Hitmonchan
   Pokémon Cristal: Gira los brazos con rapidez para <<coger>> fuerza antes de golpear. Los ataques con PUÑO son su fuerte.



# Vectorización

In [18]:
import nltk

# 1. Descarga y carga de stop-words en español
nltk.download('stopwords')  
from nltk.corpus import stopwords
spanish_stop = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to C:\Users\Juanlu
[nltk_data]     hijo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [27]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Prepara las stop-words en español
nltk.download('stopwords')
spanish_stop = stopwords.words('spanish')

# 2. Conéctate a Elasticsearch y extrae la "última" descripción de cada documento
docs = []
for hit in helpers.scan(es, index=index_name, query={"query": {"match_all": {}}}):
    descs = hit["_source"].get("descriptions", {})
    if isinstance(descs, dict) and descs:
        # Tomamos el valor de la última clave del dict (mantiene orden de inserción)
        last_desc = list(descs.values())[-1]
    else:
        last_desc = ""
    docs.append(last_desc)

print(f"Total documentos extraídos: {len(docs)}")
# Opcional: ver un par de ejemplos
for i, doc in enumerate(docs[:3]):
    print(f"Doc {i}, len={len(doc)}: {doc[:60]}…")

# 3. Vectorizador TF-IDF (igual que en prácticas)
vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words=spanish_stop,
    min_df=1   # arrancamos con mínimo 1 documento; luego subiremos a 10
)

# 4. Ajuste y transformación
X = vectorizer.fit_transform(docs)

# 5. Inspección rápida de la matriz y el vocabulario
print(f"Matriz X: {X.shape[0]} documentos × {X.shape[1]} términos")

Total documentos extraídos: 501
Doc 0, len=153: A Bulbasaur es fácil verle echándose una siesta al sol. La s…
Doc 1, len=213: Este Pokémon lleva un bulbo en el lomo y, para poder con su …
Doc 2, len=219: Venusaur tiene una flor enorme en el lomo que, según parece,…
Matriz X: 501 documentos × 2983 términos
Primeros 10 términos del vocabulario: ['000' '100' '1000' '12' '18' '20' '200' '2000' '300' '360']


[nltk_data] Downloading package stopwords to C:\Users\Juanlu
[nltk_data]     hijo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
